<a href="https://colab.research.google.com/github/ni-metre/Colab_Triplet_Loss_Backup/blob/master/New_Nonet_maker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Set up gspread
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

In [2]:
% xmode verbose

Exception reporting mode: Verbose


In [2]:

#Mount the drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Force a remount of the  drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [3]:
# import the necessary packages
import warnings

warnings.filterwarnings("ignore")


import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import pandas as pd
import numpy as np
import glob
import random

import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns; sns.set()




import os
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Dense, Dropout, Flatten, Convolution1D, MaxPooling1D,ZeroPadding1D
from tensorflow.keras.layers import Input, concatenate
from tensorflow.keras.layers import Input, Lambda
from tensorflow.keras.layers import GlobalAveragePooling1D
from tensorflow.keras.layers import Add, Subtract
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Activation
from numpy import array
from sklearn.decomposition import PCA


from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE
from sklearn.metrics import mean_squared_error

import datetime
import math
from tensorflow.keras.preprocessing.sequence import pad_sequences
from collections import defaultdict
#from Max_ML3 import prep_data, div_data, div_split_data, run_model, run_multi_model, run_experiment, concatenate_timeseries_and_static_features_list

import pickle
from tensorflow.keras.backend import sum as ksum
from tensorflow.keras.backend import square as ksquare
from tensorflow.keras.backend import mean as kmean
from tensorflow.keras.backend import maximum as kmaximum
from tensorflow.keras.backend import l2_normalize
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.models import load_model
from tensorflow.keras.losses import MeanSquaredError
import time






## 0) Import Metadata

In [96]:
test_db_df=pd.read_csv('/content/drive/My Drive/Fake_Test_Log_for_Minimal_ML_med_Ac.csv')
test_db_df.head()

,File,Raw_data,Test_name,H2O,CO2,Total CO2,Acetone,Amb_Temp,Other,Instr,Sensor,Date,Time,Time_Btwn_s,Test_that_day,Rel_Hum,EtOH_temp,Instr_Temp,Instr_T2,Instr_T3,FLAG,Notes,Sample_rate,Data_pts,Baseline,New_Baseline,Baseline_Dip,Peak_neg,Peak_neg_t,Peak_neg_t_ix,Cross_t,Cross_t_ix,Peak_pos,Peak_pos_t,Peak_pos_t_ix,Area_neg_raw,Area_pos_raw,Area_neg,Area_pos_X,Area_total_X,...,Baseline_slope,slope_calc,flag_slope,a_,b_,c_,del_a,del_b,del_c,cpn_time,cpn_mV,cpn_manual_mV,cpn_manual_time,solve,solve2,del_pnt,del_ppt,del_xtime,del_cpnt,del_pn,del_pp,slope_pos_peak,slope_pos_peak_t,slope_pos_peak_t_ix,slope_neg_peak,slope_neg_peak_t,slope_neg_peak_t_ix,slope_cross_t,slope_cross_t_ix,dx2_pos_peak,dx2_pos_peak_t,dx2_pos_peak_t_ix,dx2_neg_peak,dx2_neg_peak_t,dx2_neg_peak_t_ix,dx2_cross_t,dx2_cross_t_ix,Area_pos_old,Tests,File_num
0,9600,pdata9600,4% CO2 through plain water,10.0,40000.0,0.4,0.0000,NaN,0.0,1.0,19.0,2020-08-12,4:52:10,100000.0,1.0,40.21,NaN,26.97,26.70,4.29,NaN,NaN,250.0,29739.0,138.590360,140.299732,1.7094,-201.494212,2.564,641.0,9.720,2430.0,21.255789,15.180,3795.0,NaN,NaN,-964.480165,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9600.0,NaN
1,9603,pdata9603,4% CO2 through plain water,10.0,40000.0,0.8,0.0000,NaN,0.0,1.0,19.0,2020-08-12,5:02:22,612.0,2.0,41.68,NaN,26.38,26.17,4.38,NaN,NaN,250.0,29762.0,138.598053,139.051119,0.4531,-214.762738,2.588,647.0,9.784,2446.0,22.320042,15.192,3798.0,NaN,NaN,-1031.707572,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9603.0,NaN
2,9606,pdata9606,6% CO2 through plain water,10.0,60000.0,1.4,0.0000,NaN,0.0,1.0,19.0,2020-08-12,5:11:17,535.0,3.0,43.80,NaN,25.74,25.51,4.29,NaN,NaN,250.0,29775.0,138.543365,139.034170,0.4908,-319.336814,2.584,646.0,9.896,2474.0,32.715501,15.228,3807.0,NaN,NaN,-1551.996551,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9606.0,NaN
3,9609,pdata9609,6% CO2 through plain water,10.0,60000.0,2.0,0.0000,NaN,0.0,1.0,19.0,2020-08-12,5:20:20,543.0,4.0,43.88,NaN,25.39,25.10,4.24,NaN,NaN,250.0,29766.0,138.240113,138.832362,0.5922,-321.963345,2.616,654.0,9.956,2489.0,33.050086,15.388,3847.0,NaN,NaN,-1574.392437,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9609.0,NaN
4,9612,pdata9612,4% CO2 through 1.5625 ppm acetone simulator,10.0,40000.0,2.4,1.5625,NaN,0.0,1.0,19.0,2020-08-12,5:52:40,1940.0,5.0,40.49,NaN,26.04,25.87,4.27,NaN,NaN,250.0,29744.0,138.525839,138.691733,0.1659,-175.652029,2.588,647.0,9.692,2423.0,19.664613,15.152,3788.0,NaN,NaN,-843.821572,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9612.0,NaN


In [97]:
test_db_df.head()

,File,Raw_data,Test_name,H2O,CO2,Total CO2,Acetone,Amb_Temp,Other,Instr,Sensor,Date,Time,Time_Btwn_s,Test_that_day,Rel_Hum,EtOH_temp,Instr_Temp,Instr_T2,Instr_T3,FLAG,Notes,Sample_rate,Data_pts,Baseline,New_Baseline,Baseline_Dip,Peak_neg,Peak_neg_t,Peak_neg_t_ix,Cross_t,Cross_t_ix,Peak_pos,Peak_pos_t,Peak_pos_t_ix,Area_neg_raw,Area_pos_raw,Area_neg,Area_pos_X,Area_total_X,...,Baseline_slope,slope_calc,flag_slope,a_,b_,c_,del_a,del_b,del_c,cpn_time,cpn_mV,cpn_manual_mV,cpn_manual_time,solve,solve2,del_pnt,del_ppt,del_xtime,del_cpnt,del_pn,del_pp,slope_pos_peak,slope_pos_peak_t,slope_pos_peak_t_ix,slope_neg_peak,slope_neg_peak_t,slope_neg_peak_t_ix,slope_cross_t,slope_cross_t_ix,dx2_pos_peak,dx2_pos_peak_t,dx2_pos_peak_t_ix,dx2_neg_peak,dx2_neg_peak_t,dx2_neg_peak_t_ix,dx2_cross_t,dx2_cross_t_ix,Area_pos_old,Tests,File_num
0,9600,pdata9600,4% CO2 through plain water,10.0,40000.0,0.4,0.0000,NaN,0.0,1.0,19.0,2020-08-12,4:52:10,100000.0,1.0,40.21,NaN,26.97,26.70,4.29,NaN,NaN,250.0,29739.0,138.590360,140.299732,1.7094,-201.494212,2.564,641.0,9.720,2430.0,21.255789,15.180,3795.0,NaN,NaN,-964.480165,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9600.0,NaN
1,9603,pdata9603,4% CO2 through plain water,10.0,40000.0,0.8,0.0000,NaN,0.0,1.0,19.0,2020-08-12,5:02:22,612.0,2.0,41.68,NaN,26.38,26.17,4.38,NaN,NaN,250.0,29762.0,138.598053,139.051119,0.4531,-214.762738,2.588,647.0,9.784,2446.0,22.320042,15.192,3798.0,NaN,NaN,-1031.707572,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9603.0,NaN
2,9606,pdata9606,6% CO2 through plain water,10.0,60000.0,1.4,0.0000,NaN,0.0,1.0,19.0,2020-08-12,5:11:17,535.0,3.0,43.80,NaN,25.74,25.51,4.29,NaN,NaN,250.0,29775.0,138.543365,139.034170,0.4908,-319.336814,2.584,646.0,9.896,2474.0,32.715501,15.228,3807.0,NaN,NaN,-1551.996551,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9606.0,NaN
3,9609,pdata9609,6% CO2 through plain water,10.0,60000.0,2.0,0.0000,NaN,0.0,1.0,19.0,2020-08-12,5:20:20,543.0,4.0,43.88,NaN,25.39,25.10,4.24,NaN,NaN,250.0,29766.0,138.240113,138.832362,0.5922,-321.963345,2.616,654.0,9.956,2489.0,33.050086,15.388,3847.0,NaN,NaN,-1574.392437,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9609.0,NaN
4,9612,pdata9612,4% CO2 through 1.5625 ppm acetone simulator,10.0,40000.0,2.4,1.5625,NaN,0.0,1.0,19.0,2020-08-12,5:52:40,1940.0,5.0,40.49,NaN,26.04,25.87,4.27,NaN,NaN,250.0,29744.0,138.525839,138.691733,0.1659,-175.652029,2.588,647.0,9.692,2423.0,19.664613,15.152,3788.0,NaN,NaN,-843.821572,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9612.0,NaN


In [98]:
#Convert to numeric to enable test sorting/picking
for column in test_db_df[['Acetone', 'CO2', 'H2O', 'Sensor', 'Test_that_day', 'Time_Btwn_s', 'Instr_Temp', 'Baseline', 'File', 'Total CO2']]:
    test_db_df[column] = pd.to_numeric(test_db_df[column])
test_db_df.set_index('File', inplace = True)
test_db_df.head()

,Raw_data,Test_name,H2O,CO2,Total CO2,Acetone,Amb_Temp,Other,Instr,Sensor,Date,Time,Time_Btwn_s,Test_that_day,Rel_Hum,EtOH_temp,Instr_Temp,Instr_T2,Instr_T3,FLAG,Notes,Sample_rate,Data_pts,Baseline,New_Baseline,Baseline_Dip,Peak_neg,Peak_neg_t,Peak_neg_t_ix,Cross_t,Cross_t_ix,Peak_pos,Peak_pos_t,Peak_pos_t_ix,Area_neg_raw,Area_pos_raw,Area_neg,Area_pos_X,Area_total_X,Area_pos,...,Baseline_slope,slope_calc,flag_slope,a_,b_,c_,del_a,del_b,del_c,cpn_time,cpn_mV,cpn_manual_mV,cpn_manual_time,solve,solve2,del_pnt,del_ppt,del_xtime,del_cpnt,del_pn,del_pp,slope_pos_peak,slope_pos_peak_t,slope_pos_peak_t_ix,slope_neg_peak,slope_neg_peak_t,slope_neg_peak_t_ix,slope_cross_t,slope_cross_t_ix,dx2_pos_peak,dx2_pos_peak_t,dx2_pos_peak_t_ix,dx2_neg_peak,dx2_neg_peak_t,dx2_neg_peak_t_ix,dx2_cross_t,dx2_cross_t_ix,Area_pos_old,Tests,File_num
File,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
9600,pdata9600,4% CO2 through plain water,10.0,40000.0,0.4,0.0000,NaN,0.0,1.0,19.0,2020-08-12,4:52:10,100000.0,1.0,40.21,NaN,26.97,26.70,4.29,NaN,NaN,250.0,29739.0,138.590360,140.299732,1.7094,-201.494212,2.564,641.0,9.720,2430.0,21.255789,15.180,3795.0,NaN,NaN,-964.480165,NaN,NaN,531.85,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9600.0,NaN
9603,pdata9603,4% CO2 through plain water,10.0,40000.0,0.8,0.0000,NaN,0.0,1.0,19.0,2020-08-12,5:02:22,612.0,2.0,41.68,NaN,26.38,26.17,4.38,NaN,NaN,250.0,29762.0,138.598053,139.051119,0.4531,-214.762738,2.588,647.0,9.784,2446.0,22.320042,15.192,3798.0,NaN,NaN,-1031.707572,NaN,NaN,553.44,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9603.0,NaN
9606,pdata9606,6% CO2 through plain water,10.0,60000.0,1.4,0.0000,NaN,0.0,1.0,19.0,2020-08-12,5:11:17,535.0,3.0,43.80,NaN,25.74,25.51,4.29,NaN,NaN,250.0,29775.0,138.543365,139.034170,0.4908,-319.336814,2.584,646.0,9.896,2474.0,32.715501,15.228,3807.0,NaN,NaN,-1551.996551,NaN,NaN,863.32,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9606.0,NaN
9609,pdata9609,6% CO2 through plain water,10.0,60000.0,2.0,0.0000,NaN,0.0,1.0,19.0,2020-08-12,5:20:20,543.0,4.0,43.88,NaN,25.39,25.10,4.24,NaN,NaN,250.0,29766.0,138.240113,138.832362,0.5922,-321.963345,2.616,654.0,9.956,2489.0,33.050086,15.388,3847.0,NaN,NaN,-1574.392437,NaN,NaN,854.43,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9609.0,NaN
9612,pdata9612,4% CO2 through 1.5625 ppm acetone simulator,10.0,40000.0,2.4,1.5625,NaN,0.0,1.0,19.0,2020-08-12,5:52:40,1940.0,5.0,40.49,NaN,26.04,25.87,4.27,NaN,NaN,250.0,29744.0,138.525839,138.691733,0.1659,-175.652029,2.588,647.0,9.692,2423.0,19.664613,15.152,3788.0,NaN,NaN,-843.821572,NaN,NaN,436.60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9612.0,NaN


In [99]:
test_db_df.head(-1)

,Raw_data,Test_name,H2O,CO2,Total CO2,Acetone,Amb_Temp,Other,Instr,Sensor,Date,Time,Time_Btwn_s,Test_that_day,Rel_Hum,EtOH_temp,Instr_Temp,Instr_T2,Instr_T3,FLAG,Notes,Sample_rate,Data_pts,Baseline,New_Baseline,Baseline_Dip,Peak_neg,Peak_neg_t,Peak_neg_t_ix,Cross_t,Cross_t_ix,Peak_pos,Peak_pos_t,Peak_pos_t_ix,Area_neg_raw,Area_pos_raw,Area_neg,Area_pos_X,Area_total_X,Area_pos,...,Baseline_slope,slope_calc,flag_slope,a_,b_,c_,del_a,del_b,del_c,cpn_time,cpn_mV,cpn_manual_mV,cpn_manual_time,solve,solve2,del_pnt,del_ppt,del_xtime,del_cpnt,del_pn,del_pp,slope_pos_peak,slope_pos_peak_t,slope_pos_peak_t_ix,slope_neg_peak,slope_neg_peak_t,slope_neg_peak_t_ix,slope_cross_t,slope_cross_t_ix,dx2_pos_peak,dx2_pos_peak_t,dx2_pos_peak_t_ix,dx2_neg_peak,dx2_neg_peak_t,dx2_neg_peak_t_ix,dx2_cross_t,dx2_cross_t_ix,Area_pos_old,Tests,File_num
File,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
9600,pdata9600,4% CO2 through plain water,10.0,40000.0,0.400000,0.0000,NaN,0.0,1.0,19.0,2020-08-12,4:52:10,100000.000000,1.000000,40.21,NaN,26.970000,26.70,4.29,NaN,NaN,250.0,29739.0,138.590360,140.299732,1.7094,-201.494212,2.564,641.0,9.720,2430.0,21.255789,15.180,3795.0,NaN,NaN,-964.480165,NaN,NaN,531.85,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9600.0,NaN
9603,pdata9603,4% CO2 through plain water,10.0,40000.0,0.800000,0.0000,NaN,0.0,1.0,19.0,2020-08-12,5:02:22,612.000000,2.000000,41.68,NaN,26.380000,26.17,4.38,NaN,NaN,250.0,29762.0,138.598053,139.051119,0.4531,-214.762738,2.588,647.0,9.784,2446.0,22.320042,15.192,3798.0,NaN,NaN,-1031.707572,NaN,NaN,553.44,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9603.0,NaN
9606,pdata9606,6% CO2 through plain water,10.0,60000.0,1.400000,0.0000,NaN,0.0,1.0,19.0,2020-08-12,5:11:17,535.000000,3.000000,43.80,NaN,25.740000,25.51,4.29,NaN,NaN,250.0,29775.0,138.543365,139.034170,0.4908,-319.336814,2.584,646.0,9.896,2474.0,32.715501,15.228,3807.0,NaN,NaN,-1551.996551,NaN,NaN,863.32,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9606.0,NaN
9609,pdata9609,6% CO2 through plain water,10.0,60000.0,2.000000,0.0000,NaN,0.0,1.0,19.0,2020-08-12,5:20:20,543.000000,4.000000,43.88,NaN,25.390000,25.10,4.24,NaN,NaN,250.0,29766.0,138.240113,138.832362,0.5922,-321.963345,2.616,654.0,9.956,2489.0,33.050086,15.388,3847.0,NaN,NaN,-1574.392437,NaN,NaN,854.43,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9609.0,NaN
9612,pdata9612,4% CO2 through 1.5625 ppm acetone simulator,10.0,40000.0,2.400000,1.5625,NaN,0.0,1.0,19.0,2020-08-12,5:52:40,1940.000000,5.000000,40.49,NaN,26.040000,25.87,4.27,NaN,NaN,250.0,29744.0,138.525839,138.691733,0.1659,-175.652029,2.588,647.0,9.692,2423.0,19.664613,15.152,3788.0,NaN,NaN,-843.821572,NaN,NaN,436.60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9612.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38246,NaN,Combo of 9741 and 9810 | Weight - 0.7931034482...,10.0,60000.0,33.041379,25.0000,NaN,NaN,1.0,19.0,NaN,NaN,539.206897,66.241379,NaN,NaN,31.042069,NaN,NaN,NaN,NaN,250.0,29782.0,142.264926,143.389074,NaN,-382.423520,2.120,530.0,7.492,1873.0,74.482917,12.516,3129.0,NaN,NaN,-1516.000000,NaN,NaN,1979.65,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

#### Below enter the path name of the test_output files

In [100]:
sensor_of_interest = 19

#path to fake data
path = '/content/drive/My Drive/Minimal_Fakes_' + str(sensor_of_interest) + '_med_Ac'

pickle_path = '/content/drive/My Drive/Pickles'

#path to model checkpoints
checkpoint_path = '/content/drive/My Drive/Checkpoints'

#path to real data
real_path = '/content/drive/My Drive/Test_outputs/General'

## 1) Defining the sets of 9 tests

#### **get_test_list** returns a list of tests that meet ALL of the criteria defined in the arguments:
#### - Acetone can be a specific value, 'all', '>(value)' or '<(value)'
#### -  CO2 can be 4 (for 40000 - 46666), 5 (for 46667 - 53333), 6 (for 53334 - 60000), 'all' or '+' for CO@ values that are > 0
#### - H2O must be a specific value, always 10 for tests of interest
#### - Sensor can be a specific sensor number or 'all'
#### - Range1 and Range2 are the lower and upper limits for the range of tests of interest from (range1) to (range2 - 1)




In [101]:
def get_test_list (df, Acetone, CO2, H2O, Sensor, Range1, Range2):
    
    if Sensor == 'all':
        filtered = df
    else:
        filtered = df.query('Sensor == @Sensor')
    if Acetone != 'all':
        if str(Acetone)[:1] == '<':
            comp = float(Acetone[1:])
            filtered = filtered.query('Acetone < @comp')
        else:
            if str(Acetone)[:1] == '>':
                comp = float(Acetone[1:])
                filtered = filtered.query('Acetone > @comp')
            else:
                filtered = filtered.query('Acetone == @Acetone')
    if CO2 != 'all':
        if CO2 == '+':
            filtered = filtered.query('CO2 != 0')
        else:
            if CO2 == 4:
                filtered = filtered.query('CO2 >= 40000 & CO2 <= 46666.67')
            else:
                if CO2 == 5:
                    filtered = filtered.query('CO2 > 46666.67 & CO2 <= 53333.33')
                else:
                    filtered = filtered.query('CO2 > 53333.33 & CO2 <= 60000')
    filtered = filtered.query('H2O == @H2O')
    filtered['Tests'] = filtered.index  
    list_of_tests = filtered['Tests'].tolist()
    return([y for y in list_of_tests if ((int(y)>=Range1)&(int(y)<Range2))])
    #return(filtered)

In [10]:
#Returns a list of tests that meet ALL of the criteria defined in the arguments
# SAME AS get_test_list_Temp above for the following
# Acetone can be a specific value, 'all', '>(value)' or '<(value)'
# CO2 can be a specific value, 'all' or '+' for CO@ values that are > 0
# H2O must be a specific value, almost always 10 for tests of interest
# Sensor can be a specific sensor number or 'all'
# temp is a temperature value in degrees C
# and delta is range in degrees C above and below temp that will be included in the returned list
# BUT
# This function takes a list of tests as its argument instead of a range of tests

def get_test_sublist_Temp (Acetone, CO2, H2O, temp, delta, Sensor, testlist):

    filtered = test_db_df[test_db_df.index.isin(testlist)]

    if Sensor != 'all':
        filtered = filtered[(filtered['Sensor'] == Sensor)]

    if Acetone != 'all':
        if str(Acetone)[:1] == '<':
            filtered = filtered[filtered['Acetone'].astype(float) < float(Acetone[1:])]
        else:
            if str(Acetone)[:1] == '>':
                filtered = filtered[filtered['Acetone'].astype(float) > float(Acetone[1:])]
            else:
                filtered = filtered[(filtered['Acetone'] == Acetone)]
    if CO2 != 'all':
        if CO2 == '+':
            filtered = filtered[(filtered['CO2'] != '0')]
        else:
            filtered = filtered[(filtered['CO2'] == CO2)]
    filtered = filtered[(filtered['H2O'] == H2O)]
    filtered['Tests'] = filtered.index
    filtered['Instr_Temp'] = pd.to_numeric(filtered['Instr_Temp'])
    filtered = filtered[filtered['Instr_Temp']>= temp - delta]
    filtered = filtered[filtered['Instr_Temp']<= temp + delta]
    ace_vals = filtered['Acetone'].unique()
    CO2_vals = filtered['CO2'].unique()
    list_of_tests = filtered['Tests'].tolist()
    return list_of_tests

In [102]:
global Ace_close, Ace_far, CO2_close, CO2_far, Temp_close, Temp_far

Ace_close = 2
Ace_far = 5
CO2_close = 2000
CO2_far = 5000
Temp_close = 2
Temp_far = 5

def get_nonet (anchor, near_ace, near_co2, near_temp, far_ace, far_co2, anchor2):
    anchor_ace = subset_df.loc[anchor, 'Acetone']
    anchor_co = subset_df.loc[anchor, 'CO2']
    anchor_temp = subset_df.loc[anchor, 'Instr_Temp']
    filtered = subset_df
    
    if near_ace:
        complow = float(anchor_ace) - Ace_close
        comphigh = float(anchor_ace) + Ace_close
        filtered = filtered.query('Acetone >= @complow and Acetone <= @comphigh')
    else:
        complow = float(anchor_ace) - Ace_far
        comphigh = float(anchor_ace) + Ace_far
        filtered = filtered.query('Acetone < @complow or Acetone > @comphigh')
    if near_co2:
        complow = float(anchor_co) - CO2_close
        comphigh = float(anchor_co) + CO2_close
        filtered = filtered.query('CO2 >= @complow and CO2 <= @comphigh')
    else:
        complow = float(anchor_co) - CO2_far
        comphigh = float(anchor_co) + CO2_far
        filtered = filtered.query('CO2 < @complow or CO2 > @comphigh')
    if near_temp:
        complow = float(anchor_temp) - Temp_close
        comphigh = float(anchor_temp) + Temp_close
        filtered = filtered.query('Instr_Temp >= @complow and Instr_Temp <= @comphigh')
    else:
        complow = float(anchor_temp) - Temp_far
        comphigh = float(anchor_temp) + Temp_far
        filtered = filtered.query('Instr_Temp < @complow or Instr_Temp > @comphigh')
    if far_ace:
        anchor2_ace = test_db_df.loc[anchor2, 'Acetone']
        complow = float(anchor2_ace) - Ace_far
        comphigh = float(anchor2_ace) + Ace_far
        filtered = filtered.query('Acetone < @complow or Acetone > @comphigh')
    if far_co2:
        anchor2_co = test_db_df.loc[anchor2, 'CO2']
        complow = float(anchor2_co) - CO2_far
        comphigh = float(anchor2_co) + CO2_far
        filtered = filtered.query('CO2 < @complow or CO2 > @comphigh')
        
    filtered['Tests'] = filtered.index  
    list_of_tests = filtered['Tests'].tolist()
    return(list_of_tests)

In [103]:
test_set = get_test_list (test_db_df, 'all', '+', 10, sensor_of_interest, 10000, 1905000)

subset_df = test_db_df.loc[test_set]

#  get a random test and remove it from the list
#  get_nonet (anchor, near_ace, near_co2, near_temp, far_ace, far_co2, anchor2)

print(len(subset_df))

28252


## IF YOU WANT TO READ FROM A PICKLED VERSION OF THE 1000 NONTETS (PREVIOUSLY GENERATED 08-07-2020) SKIP TO THE SECTION **"START HERE TO READ IN A PICKLE VERSION OF ttvdict!!!!!""**


#### Code here does the picking of the nonets based on the distances defined by **Ace_close, Ace_far, CO2_close, CO2_far, Temp_close,** and **Temp_far**

#### Acetone values range from 0 to 100 ppm, CO2 values range from 40000 to 60000 ppm, and temperature values range from about 7C to 39C.

In [ ]:
#  get a random test and remove it from the list
#  get_nonet (anchor, near_ace, near_co2, near_temp, far_ace, far_co2, anchor2)


c1_list = []
c2_list = []
c3_list = []
c4_list = []
c5_list = []
c6_list = []
c7_list = []
c8_list = []
c9_list = []


##### You can set the number of 
for i in range(0,5000):
    try:
        print(i)
        A = random.choice(test_set)
        B = random.choice(get_nonet (A, True, False, True, False, False, 0))
        C = random.choice(get_nonet (A, False, False, True, False, True, B))
        D = random.choice(get_nonet (C, False, True, True, True, False, B))
        E = random.choice(get_nonet (A, False, True, True, False, False, 0))
        F = random.choice(get_nonet (A, False, True, False, True, False, E))
        G = random.choice(get_nonet (A, True, False, False, False, True, B))
        while True:
            H = random.choice(get_nonet (A, True, True, True, False, False, 0))
            if H != A:
                break
        I = random.choice(get_nonet (A, True, True, False, False, False, 0))
        c1_list.append(A)
        c2_list.append(B)
        c3_list.append(C)
        c4_list.append(D)
        c5_list.append(E)
        c6_list.append(F)
        c7_list.append(G)
        c8_list.append(H)
        c9_list.append(I)
    except IndexError:
        continue

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [94]:
#Turn lists into dictionary
cdict = {'c1_list': c1_list, 'c2_list': c2_list, 'c3_list': c3_list, 'c4_list': c4_list, 'c5_list': c5_list, 'c6_list': c6_list, 'c7_list': c7_list, 'c8_list': c8_list,'c9_list': c9_list}
print(len(cdict['c1_list']))

223


In [95]:
with open(pickle_path +'/cdict_med_Ac_223.pickle', 'wb') as handle:
    pickle.dump(cdict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [92]:
load_list = [177, 293, 187, 163, 300, 123, 145, 117, 134, 400, 149, 194, 147, 234, 116,111, 117,223]
print(np.sum(load_list))

3107


In [ ]:
dict_list = []
load_list = [177, 293, 187, 163, 300, 123, 145]
print(np.sum(load_list))

for entry in load_list:
  with open(pickle_path +'/cdict_19D_' + str(entry) + '.pickle', 'rb') as handle:
    cdict = pickle.load(handle)
    dict_list.append(cdict)
print(len(dict_list))

# the logic
#for k, v in ((k, v) for d in dicts for k, v in d.iteritems()):
#   final[k].append(v)

5000
7


In [ ]:
cdict = defaultdict(list)
for d in dict_list: # you can list as many input dicts as you want here
    for key, value in d.items():
        cdict[key].extend(value)
print(cdict.keys())
print(len(cdict['c1_list']))

dict_keys(['c1_list', 'c2_list', 'c3_list', 'c4_list', 'c5_list', 'c6_list', 'c7_list', 'c8_list', 'c9_list'])
5000


## 2) Reading in and preprocessing the data

#### Code below reads in csvs (signal data), acetone, co2 , and temperature data for each test in the 9 groups.

#### In this code, CO2 values are scaled by 2000 so they are within the same range as the acetone values. There is an option to change this scaling factor below (see **RESCALE DATA**)

In [ ]:
#with open(pickle_path +'/cdict_15_metadata_10000.pickle', 'rb') as handle:
    
 #   cdict = pickle.load(handle)


In [ ]:
#This usually takes 60 - 90 minutes to run 1000 nonets

scale = 1
cut_off_time =30

filenumdict = defaultdict(list)
testdict = defaultdict(list)
Acdict =defaultdict(list)
co2dict = defaultdict(list)
tempdict = defaultdict(list)
metadatadict = defaultdict(list)
ppdict = defaultdict(list)
pptdict = defaultdict(list)
pndict = defaultdict(list)
pntdict = defaultdict(list)


for i in range(1, len(cdict)+1):
    print(i)
    filenum_list = []
    test_list = []
    Ac_list = []
    co2_list = []
    temp_list = []
    metadata_list = []
    pp_list = []
    ppt_list = []
    pn_list = []
    pnt_list = []
    for j in cdict['c' + str(i) + '_list']:
            ac = float(test_db_df.loc[j, 'Acetone'])
            co2 = int(test_db_df.loc[j, 'CO2'])/2000
            temp = float(test_db_df.loc[j, 'Instr_Temp'])
            sample_rate = int(test_db_df.loc[j, 'Sample_rate'])
            num_of_samples = int(cut_off_time*sample_rate/scale)
            bs = float(test_db_df.loc[j, 'Baseline'])
            timebtwn = float(test_db_df.loc[j, 'Time_Btwn_s'])
            totalco2 = float(test_db_df.loc[j, 'Total CO2'])
            testday = float(test_db_df.loc[j, 'Test_that_day'])
            pp =  float(test_db_df.loc[j, 'Peak_pos'])
            ppt =  float(test_db_df.loc[j, 'Peak_pos_t'])
            pn =  float(test_db_df.loc[j, 'Peak_neg'])
            pnt =  float(test_db_df.loc[j, 'Peak_neg_t'])

        
            full_data = pd.read_csv(path+ '/test_output' + str(int(j)) + '.csv',header = 0)
            sig_data = full_data[['Second','mV_rolling_mean']].truncate(after = cut_off_time*sample_rate-1)
            mV_data = sig_data['mV_rolling_mean'].to_numpy()
            mV_data[np.isnan(mV_data)]=0
            # All the fake data is currently at 250 Hz so no recasting of the data is needed       
            #if sample_rate == 500:
            #    mV_data = mV_data.reshape(-1, 2).mean(axis=1)
            filenum_list.append(j)
            test_list.append(mV_data)
            Ac_list.append(ac)
            co2_list.append(co2)
            temp_list.append(temp)
            metadata_list.append([temp, bs, timebtwn, totalco2, testday])
            pp_list.append(pp)
            ppt_list.append(ppt)
            pn_list.append(pn)
            pnt_list.append(pnt)
    filenumdict['c' + str(i)+'num'] = np.array(filenum_list)
    testdict['c' + str(i)+'tests'] = np.array(test_list)
    Acdict['c' + str(i) + 'Ac']= np.array(Ac_list)
    co2dict['c' + str(i) + 'co2']= np.array(co2_list)
    tempdict['c' + str(i) + 'temp']= np.array(temp_list)
    metadatadict['c' + str(i)+'metadata'] = np.array(metadata_list)
    ppdict['c' + str(i)+'peak_pos'] = np.array(pp_list)
    pptdict['c' + str(i)+'peak_pos_t'] = np.array(ppt_list)
    pndict['c' + str(i)+'peak_neg'] = np.array(pn_list)
    pntdict['c' + str(i)+'peak_neg_t'] = np.array(pnt_list)
    

1
2


In [ ]:


print(len(metadatadict['c8metadata']))

5000


In [ ]:
foo = pd.read_csv('/content/drive/My Drive/Fakes_19/test_output835697.csv')
print(foo)

        Second  mV_rolling_mean
0        0.000        -0.575590
1        0.004        -0.577519
2        0.008        -0.579568
3        0.012        -0.578951
4        0.016        -0.580697
...        ...              ...
29712  118.848         2.181615
29713  118.852         2.179910
29714  118.856         2.180175
29715  118.860         2.179836
29716  118.864         2.180924

[29717 rows x 2 columns]


#### Tuns lists into arrays and front-zero-pads the csvs (signal data) so that the csvs are all the same length

In [ ]:
#Check for the csv with the longest length
lengths = []
for i in range(0, len(testdict['c1tests'])-1):
    for j in range(1,5):
        lengths.append(len(testdict['c' +str(j)+'tests'][i]))

maxlen = max(lengths)
print(maxlen)

7500


In [ ]:
#Front pad the csvs so they are the same length as the longest csv (maxlen)
#Turn lists into arrays

paddict= defaultdict(list)
for i in range(1, len(cdict)+1):
    paddict['paddedc' + str(i)] = pad_sequences(testdict['c' +str(i) + 'tests'], maxlen = maxlen)
    print(paddict['paddedc'+str(i)].shape)


(5000, 7500)
(5000, 7500)
(5000, 7500)
(5000, 7500)
(5000, 7500)
(5000, 7500)
(5000, 7500)
(5000, 7500)
(5000, 7500)


In [ ]:
# Combine everything to a dictionary
ttvdict = dict(paddict, **Acdict)
ttvdict.update(co2dict)
ttvdict.update(tempdict)
ttvdict.update(metadatadict)

indict = {'indices': range(len(ttvdict['paddedc1']))}
ttvdict.update(indict)
print(ttvdict.keys())
print(len(ttvdict))

dict_keys(['paddedc1', 'paddedc2', 'paddedc3', 'paddedc4', 'paddedc5', 'paddedc6', 'paddedc7', 'paddedc8', 'paddedc9', 'c1Ac', 'c2Ac', 'c3Ac', 'c4Ac', 'c5Ac', 'c6Ac', 'c7Ac', 'c8Ac', 'c9Ac', 'c1co2', 'c2co2', 'c3co2', 'c4co2', 'c5co2', 'c6co2', 'c7co2', 'c8co2', 'c9co2', 'c1temp', 'c2temp', 'c3temp', 'c4temp', 'c5temp', 'c6temp', 'c7temp', 'c8temp', 'c9temp', 'c1metadata', 'c2metadata', 'c3metadata', 'c4metadata', 'c5metadata', 'c6metadata', 'c7metadata', 'c8metadata', 'c9metadata', 'indices'])
46


In [ ]:
# To save ttvdict as a pickle
#You have ttvdict_15_metadata_2638.pickle



with open(pickle_path +'/ttvdict_19_metadata_5000_30s_.pickle', 'wb') as handle:
    pickle.dump(ttvdict, handle, protocol=pickle.HIGHEST_PROTOCOL)



In [ ]:
# Another dictionary with peak data and testnum
# Combine everything to a dictionary
ttvdict = dict(filenumdict, **paddict)
ttvdict.update(Acdict)
ttvdict.update(co2dict)
ttvdict.update(tempdict)
ttvdict.update(metadatadict)
ttvdict.update(ppdict)
ttvdict.update(pptdict)
ttvdict.update(pndict)
ttvdict.update(pntdict)

indict = {'indices': range(len(ttvdict['paddedc1']))}
ttvdict.update(indict)
print(ttvdict.keys())
print(len(ttvdict))

dict_keys(['c1num', 'c2num', 'c3num', 'c4num', 'c5num', 'c6num', 'c7num', 'c8num', 'c9num', 'paddedc1', 'paddedc2', 'paddedc3', 'paddedc4', 'paddedc5', 'paddedc6', 'paddedc7', 'paddedc8', 'paddedc9', 'c1Ac', 'c2Ac', 'c3Ac', 'c4Ac', 'c5Ac', 'c6Ac', 'c7Ac', 'c8Ac', 'c9Ac', 'c1co2', 'c2co2', 'c3co2', 'c4co2', 'c5co2', 'c6co2', 'c7co2', 'c8co2', 'c9co2', 'c1temp', 'c2temp', 'c3temp', 'c4temp', 'c5temp', 'c6temp', 'c7temp', 'c8temp', 'c9temp', 'c1metadata', 'c2metadata', 'c3metadata', 'c4metadata', 'c5metadata', 'c6metadata', 'c7metadata', 'c8metadata', 'c9metadata', 'c1peak_pos', 'c2peak_pos', 'c3peak_pos', 'c4peak_pos', 'c5peak_pos', 'c6peak_pos', 'c7peak_pos', 'c8peak_pos', 'c9peak_pos', 'c1peak_pos_t', 'c2peak_pos_t', 'c3peak_pos_t', 'c4peak_pos_t', 'c5peak_pos_t', 'c6peak_pos_t', 'c7peak_pos_t', 'c8peak_pos_t', 'c9peak_pos_t', 'c1peak_neg', 'c2peak_neg', 'c3peak_neg', 'c4peak_neg', 'c5peak_neg', 'c6peak_neg', 'c7peak_neg', 'c8peak_neg', 'c9peak_neg', 'c1peak_neg_t', 'c2peak_neg_t', 'c3

In [ ]:
# To save ttvdict as a pickle
#You have ttvdict_15_metadata_2638.pickle



with open(pickle_path +'/ttvdict_16_csvs_sensor_metadata_peaks.pickle', 'wb') as handle:
    pickle.dump(ttvdict, handle, protocol=pickle.HIGHEST_PROTOCOL)